# **Data Preprocessing**
**DO NOT RUN**

In [ ]:
# @markdown # **INIT**
!curl -fsSL https://ollama.com/install.sh | sh

import threading
import subprocess
import time
def run():
  def run_ollama_serve():
    subprocess.Popen(["ollama", "serve"])

  thread = threading.Thread(target=run_ollama_serve)
  thread.start()
  time.sleep(5)

from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)


run()
!ollama pull llama3

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 

In [ ]:
#@markdown # DO NOT RUN, RATHER UPLOAD CSV
import os
import zipfile
import json
import pandas as pd
import subprocess
import re

# Define paths
zip_file_path = "/content/resumes.zip"       # Path to your zip file of JSONs
json_extract_dir = "/content/json_files/"       # Temporary directory for extracted JSON files
resume_dir = "/content/resume"                 # Directory to save cleaned resume texts
output_dir = "/content/outputs"                # Directory to save ollama outputs

# Create necessary directories if they don't exist
os.makedirs(json_extract_dir, exist_ok=True)
os.makedirs(resume_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

# 1. Extract the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(json_extract_dir)


json_extract_dir = "/content/json_files/ResumesJsonAnnotated/"       # Temporary directory for extracted JSON files

# 2. Define a function to clean text:
def clean_text(text):
    # Remove newlines
    text = text.replace("\n", " ")
    # Remove stray characters by keeping only ASCII characters
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    # Optionally, you can remove extra spaces:
    return " ".join(text.split())

# 3. Process each JSON file: extract 'text', clean it, and save to /resume folder.
# Also, collect data for the DataFrame.
data = []  # List of dictionaries for each row in DataFrame

json_files = [f for f in os.listdir(json_extract_dir) if f.endswith(".json")]

for json_file in json_files:
    json_path = os.path.join(json_extract_dir, json_file)
    with open(json_path, "r", encoding="utf-8") as f:
        try:
            content = json.load(f)
        except json.JSONDecodeError:
            print(f"Error reading {json_file}. Skipping.")
            continue

    # Get the text attribute and clean it
    raw_text = content.get("text", "")
    cleaned_text = clean_text(raw_text)

    # Save cleaned text to resume directory (name based on JSON file)
    base_name = os.path.splitext(json_file)[0]
    match = re.search(r"\((\d+)\)", base_name)
    resume_file_path = os.path.join(resume_dir, f"{match.group(1)}.txt")
    with open(resume_file_path, "w", encoding="utf-8") as f:
        f.write(cleaned_text)

    # Add a row for the DataFrame with annotations left empty for now
    data.append({
        "json_file_name": json_file,
        "resume_text": cleaned_text,
        "annotations": ""  # Placeholder for later
    })

for entry in data:
  # Extract the number from json_file_name using regex
  match = re.search(r"\((\d+)\)", entry["json_file_name"])
  if match:
      # Replace json_file_name with the extracted number
      entry["json_file_name"] = int(match.group(1))  # Convert to an integer for numeric sorting

# Sort the list of dictionaries based on the updated json_file_name
data.sort(key=lambda x: x["json_file_name"])

In [ ]:
# @markdown # INGEST CSV

FILE =  "final_resume_annotations (5000)" # @param {type:"string"}
import os
import zipfile
import json
import pandas as pd
import subprocess
import re

output_dir = "/content/outputs"
os.makedirs(output_dir, exist_ok=True)


df = pd.read_csv(f'/content/{FILE}.csv', on_bad_lines='skip') #pd.DataFrame(data)
df.head()

,json_file_name,resume_text,annotations,processed_annotations
0,1,One97 Communications Limited Data Scientist Ja...,"{\n ""name"": ""One97 Communications Limited Dat...","{\n ""name"": ""One97 Communications Limited Dat..."
1,2,KEISUKE YAMAMOTO PROFILE Professional with 3 m...,"{\n ""name"": ""Keisuke Yamamoto"",\n ""qualifica...","{\n ""name"": ""Keisuke Yamamoto"",\n ""qualifica..."
2,3,Stella Thatcher ASSOCIATE MARKET ANALYST Execu...,"{\n ""name"": ""Stella Thatcher"",\n ""qualificat...","{\n ""name"": ""Stella Thatcher"",\n ""qualificat..."
3,4,PROFILE Software developer with a niche toward...,"{\n ""name"": ""Software developer with a niche ...","{\n ""name"": ""Software developer with a niche ..."
4,5,KELSEY STENS SOFTWARE DEVELOPER(MACHINE LEARNI...,"{\n ""name"": ""KELSEY STENS"",\n ""qualification...","{\n ""name"": ""KELSEY STENS"",\n ""qualification..."


In [ ]:
from google.colab import drive

#@markdown #**CORE**
run()
SKIP_ROWS = 5000 #@param
TILL = -1 #@param
SAVE_AFTER_EVERY = 100 #@param
# @markdown `SAVE_AFTER_EVERY != -1` triggers drive mount.
if TILL == -1:
  TILL = len(df)

if SAVE_AFTER_EVERY != -1:
  drive.mount('/content/drive')

def save_csv(limit):
  annotations = []
  for json_file in df["json_file_name"]:
      base_name = json_file
      output_file_path = os.path.join(output_dir, f"{base_name}.txt")
      try:
          with open(output_file_path, "r", encoding="utf-8") as f:
              annot_text = f.read().strip()
      except FileNotFoundError:
          annot_text = ""
      annotations.append(annot_text)


  def extract_json_content(text):
      match = re.search(r"{.*}", text, re.DOTALL)  # Matches content within {}
      if match:
          return match.group(0)  # Return the matched content
      return None  # Return None if no match is found

  # Process each text
  extracted_jsons_ann = [extract_json_content(text) for text in annotations]


  start_index = df[df['json_file_name'] == SKIP_ROWS].index[0]
  # Update the DataFrame with the annotations
  df.loc[start_index:, "annotations"] = extracted_jsons_ann[start_index:]


  # Optionally, save the final DataFrame to CSV
  try:
    df.to_csv(f"/content/final_resume_annotations ({limit}).csv", index=False)
    df.to_csv(f"/content/drive/MyDrive/final_resume_annotations ({limit}).csv", index=False)
    print("\n\033[1;3;32mCSV saved successfully!\033[0m")
  except:
    print("\n\033[1;31mError saving CSV.\033[0m")

# 5. For each row, supply the resume_text to ollama for NER annotation.
# The prompt includes a newline after the instruction.
# Note: Each call to 'ollama run' is blocking until the response is received.
limit = 0
for idx, row in df.iterrows():
    limit+=1
    if limit == TILL:
        break
    if idx < SKIP_ROWS:
        continue
    if SAVE_AFTER_EVERY != -1 and limit % SAVE_AFTER_EVERY == 0:
      save_csv(limit)
    # Create prompt using the resume text from DataFrame
    resume_text = row['resume_text']
    prompt = f"""
Extract the mentioned entities from the following resume text and return the output in structured JSON format.

Identify and extract ONLY the following four categories:
- `name`: Full name of the candidate.
- `qualifications`: Academic qualifications, degrees, or certifications.
- `skills`: Relevant technical and soft skills mentioned.
- `experience`: Work experience, internships, or organizations where the candidate has worked.

Important Notes:
- For `experience`, extract any information that refers to past roles or internships, even if the format is inconsistent.
- If any category is missing or unavailable, leave it as an empty list or null.

Return the output in the following JSON format without extra explanations:
{{
  "name": "",
  "qualifications": ["", "", "", ...],
  "skills": ["", "", "", ...],
  "experience": ["", "", "", ...]
}}
The JSON file must ONLY contain the above four properties.
DO NOT create any other properties in the JSON other than the above mentioned.
DO NOT create any sub-property of any property, concatenate the relevant info in the main property itself as a complete string.
because the above 4 properties can only contain strings.

Here’s the resume text:
{resume_text}
"""

    base_name = row['json_file_name']
    output_file_path = os.path.join(output_dir, f"{base_name}.txt")

    # Command to run ollama with the prompt
    command = f'ollama run llama3 "{prompt}" > "{output_file_path}"'

    print(f"Processing {row['json_file_name']} ...", end="\t")
    try:
      result = subprocess.run(command, shell=True, capture_output=True, text=True)
    except ValueError as e:
      print(f"\n\033[1;31mError: {e} - Skipping this file!\033[0m")
    if result.returncode != 0:
        print(f"\n\033[1;31mError processing {row['json_file_name']}: \033[3;31m{result.stderr}\033[0m")
    else:
        print("\033[1;42m DONE \033[0m")

Mounted at /content/drive
Processing 5007 ...	 DONE 
Processing 5008 ...	 DONE 
Processing 5009 ...	 DONE 
Processing 5010 ...	 DONE 
Processing 5011 ...	 DONE 
Processing 5012 ...	 DONE 
Processing 5013 ...	 DONE 
Processing 5014 ...	 DONE 
Processing 5015 ...	 DONE 
Processing 5016 ...	 DONE 
Processing 5017 ...	 DONE 
Processing 5018 ...	 DONE 
Processing 5019 ...	 DONE 
Processing 5020 ...	 DONE 
Processing 5021 ...	 DONE 
Processing 5022 ...	 DONE 
Processing 5023 ...	 DONE 
Processing 5024 ...	 DONE 
Processing 5025 ...	 DONE 
Processing 5026 ...	 DONE 
Processing 5027 ...	 DONE 
Processing 5028 ...	 DONE 
Processing 5029 ...	 DONE 
Processing 5030 ...	 DONE 
Processing 5031 ...	 DONE 
Processing 5032 ...	 DONE 
Processing 5033 ...	 DONE 
Processing 5034 ...	 DONE 


In [ ]:
#@markdown # MANUAL CSV SAVE
FILE =  "final_resume_annotations (5034)" # @param {type:"string"}

annotations = []
for json_file in df["json_file_name"]:
    base_name = json_file
    output_file_path = os.path.join(output_dir, f"{base_name}.txt")
    try:
        with open(output_file_path, "r", encoding="utf-8") as f:
            annot_text = f.read().strip()
    except FileNotFoundError:
        annot_text = ""
    annotations.append(annot_text)


def extract_json_content(text):
    match = re.search(r"{.*}", text, re.DOTALL)  # Matches content within {}
    if match:
        return match.group(0)  # Return the matched content
    return None  # Return None if no match is found

# Process each text
extracted_jsons_ann = [extract_json_content(text) for text in annotations]


start_index = df[df['json_file_name'] == SKIP_ROWS].index[0]
# Update the DataFrame with the annotations
df.loc[start_index:, "annotations"] = extracted_jsons_ann[start_index:]


# Optionally, save the final DataFrame to CSV
df.to_csv(f"/content/{FILE}.csv", index=False)


df.head()

,json_file_name,resume_text,annotations,processed_annotations
0,1,One97 Communications Limited Data Scientist Ja...,"{\n ""name"": ""One97 Communications Limited Dat...","{\n ""name"": ""One97 Communications Limited Dat..."
1,2,KEISUKE YAMAMOTO PROFILE Professional with 3 m...,"{\n ""name"": ""Keisuke Yamamoto"",\n ""qualifica...","{\n ""name"": ""Keisuke Yamamoto"",\n ""qualifica..."
2,3,Stella Thatcher ASSOCIATE MARKET ANALYST Execu...,"{\n ""name"": ""Stella Thatcher"",\n ""qualificat...","{\n ""name"": ""Stella Thatcher"",\n ""qualificat..."
3,4,PROFILE Software developer with a niche toward...,"{\n ""name"": ""Software developer with a niche ...","{\n ""name"": ""Software developer with a niche ..."
4,5,KELSEY STENS SOFTWARE DEVELOPER(MACHINE LEARNI...,"{\n ""name"": ""KELSEY STENS"",\n ""qualification...","{\n ""name"": ""KELSEY STENS"",\n ""qualification..."


### Compile

1. toJSON

In [ ]:
import pandas as pd
import json

i=0
indices = []
# Check for problematic rows
def check_json(row, index):
    try:
        # Attempt to parse the JSON
        if isinstance(row, str):
            json.loads(row)
    except json.JSONDecodeError as e:
        # Print error message with the row index for debugging
        print(f"❌ JSONDecodeError in row {index}: {e}")
        global i
        i+=1
        global indices
        indices.append(idx)
        print(f"Problematic JSON: {row}")

# Apply the check to each row in the 'annotations' column
for idx, row in df["annotations"].items():
    check_json(row, idx)

In [ ]:
indices

In [ ]:
df = df.drop(indices)

# Reset index (optional)
df = df.reset_index(drop=True)

2. flatten

In [ ]:
import pandas as pd

def flatten_item(item):
    """
    Flatten a dictionary or list item into a string.
    - If item is a dictionary, concatenate all key-value pairs as 'value' strings.
    - If item is a list of dictionaries, flatten each dictionary and return a list of concatenated strings.
    - If item is a list of simple strings, concatenate the list as a single string.
    - Otherwise, return the item unchanged.
    """
    if isinstance(item, dict):
        parts = []
        for key, value in item.items():
            if isinstance(value, list):
                # Flatten list and join its elements into a string
                value = ", ".join(str(flatten_item(v)) for v in value if v is not None)
            parts.append(str(value))
        return ", ".join(parts)

    elif isinstance(item, list):
        # If list contains dictionaries, flatten each dictionary
        if len(item) > 0 and isinstance(item[0], dict):
            return [flatten_item(subitem) for subitem in item]
        else:
            # Join list of strings into one string if it's a simple list
            return ", ".join(str(v) for v in item if v is not None)

    else:
        return item

# Process field and flatten list or dictionary
def process_field(field_value):
    """
    Process the value of 'experience', 'qualifications', and 'skills'.
    - If the value is a list of dictionaries, flatten each dictionary into a string.
    - If the value is a single dictionary, flatten it into a string.
    - Otherwise, return the value as is.
    """
    if isinstance(field_value, list):
        if len(field_value) > 0 and isinstance(field_value[0], dict):
            # Flatten each dictionary in the list
            return [flatten_item(item) for item in field_value]
        else:
            return field_value
    elif isinstance(field_value, dict):
        # Flatten a single dictionary into a string
        return flatten_item(field_value)
    else:
        return field_value

def process_annotations(annotation):
    """
    Process a single JSON dictionary from the 'annotations' column.
    For the keys 'experience', 'qualifications', and 'skills', apply processing as needed.
    """
    if not isinstance(annotation, dict):
        return annotation

    new_annotation = annotation.copy()
    for key in ["experience", "qualifications", "skills"]:
        if key in new_annotation:
            original = new_annotation[key]
            new_annotation[key] = process_field(new_annotation[key])
            # Debug: Show changes only if something was modified
            if new_annotation[key] != original:
                print(f"Processed '{key}': {original} -> {new_annotation[key]}")
    return new_annotation

# Assuming df is your DataFrame with the column "annotations"
# df = pd.read_csv("your_data.csv")  # Example if loading from a CSV
# List to store row numbers (indices) of affected rows
modified_rows = []

# Apply processing and track modified rows
def process_and_track(row, index):
    if isinstance(row, dict):
        processed_row = process_annotations(row)
        if processed_row != row:
            modified_rows.append(index)  # Append the row number (index)
        return processed_row
    return row

# Apply the function while keeping track of row indices
df["processed_annotations"] = df.apply(lambda x: process_and_track(x["annotations"], x.name), axis=1)

# Print the row numbers that were modified
print(f"✅ Rows affected: {modified_rows}")

✅ Rows affected: []


In [ ]:
df.iloc[0]['processed_annotations']

'{\n  "name": "One97 Communications Limited Data Scientist" and "MACY WILLIAMS",\n  "qualifications": ["B.Tech(Computer Science) from SGBAU university in 2017", "M.Tech (Computer Science Engineering) from Indian Institute of Technology (IIT), Kanpur in 2019"],\n  "skills": ["YOLO CNN Object-detection", "OCR Insights", "Java", "C++", "Python", "Machine Learning", "Algorithms", "Natural Language Processing", "Deep Learning", "Computer Vision", "Pattern Recognition", "Data Science", "Data Analysis", "Software Engineer", "Data Analyst", "C", "PySpark", "Kubeflow"],\n  "experience": ["One97 Communications Limited Data Scientist Jan 2019 to Till Date", "MACY WILLIAMS Data Scientist"]\n}'

In [ ]:
df.to_csv("/content/final_resume_annotations_procd.csv", index=False)


In [ ]:
df.drop(columns=['processed_annotations'])

,json_file_name,resume_text,annotations
0,1,One97 Communications Limited Data Scientist Ja...,"{\n ""name"": ""One97 Communications Limited Dat..."
1,2,KEISUKE YAMAMOTO PROFILE Professional with 3 m...,"{\n ""name"": ""Keisuke Yamamoto"",\n ""qualifica..."
2,3,Stella Thatcher ASSOCIATE MARKET ANALYST Execu...,"{\n ""name"": ""Stella Thatcher"",\n ""qualificat..."
3,4,PROFILE Software developer with a niche toward...,"{\n ""name"": ""Software developer with a niche ..."
4,5,KELSEY STENS SOFTWARE DEVELOPER(MACHINE LEARNI...,"{\n ""name"": ""KELSEY STENS"",\n ""qualification..."
...,...,...,...
5024,5031,Drew Hall ASSOCIATE MACHINE LEARNING ENGINEER ...,NaN
5025,5032,PROFILE Software Developer and Data Analytics ...,NaN
5026,5033,KHOLE MOORE GRADUATE RESEARCHER SKILLS Machine...,NaN
5027,5034,TIMOTHY PETERSON SOFTWARE DEVELOPER INTERN WOR...,NaN


## Final data work

In [ ]:
!pip install datasets

import pandas as pd
from sklearn.model_selection import train_test_split

df=pd.read_csv("/content/bio_tagged_output_with_new_lines.csv", low_memory=False)
df = df[['annotations','final_annotation', 'bio_tag']]
df.dropna(how='all', inplace=True)
all_data = []


df.rename(columns={"bio_tag": "BIO_tag"}, inplace=True)


print(all_data[:1])

[]


In [ ]:
for i, row in enumerate(df["BIO_tag"]):
    tokens = []
    tags = []

    # Use splitlines() to split by actual newline characters
    for line in row.strip().splitlines():
        # Each line should have exactly two parts: token and tag
        parts = line.strip().split()
        if len(parts) != 2:
            print(f"Skipping malformed line: {line}")
            continue
        token, tag = parts
        tokens.append(token)
        tags.append(tag)

    if tokens:
        all_data.append({
            "tokens": tokens,
            "ner_tags": tags,
            "annotations": df["annotations"].iloc[i],
            "final_annotation": df["final_annotation"].iloc[i]
        })

In [ ]:
df = df.dropna(subset=['BIO_tag'])

In [ ]:
print(all_data[0])

{'tokens': ['Keisuke', 'Yamamoto', 'B.Tech(Computers)', 'Web', 'Developer', 'Data', 'Analyst', 'Web', 'Development', 'Machine', 'Learning', 'Python', 'FrontEnd', 'HTML', 'Javascript', 'PHP', 'CSS', 'MySQL', 'Java', 'Data', 'Structures', 'OOP', 'Algorithms', 'C', 'SAI', 'Technologies,', 'Intern,', 'Aug', '2020', 'to', 'Till', 'Date', 'Working', 'at', 'a', 'cybersecurity', 'company', 'as', 'an', 'intern.', 'Working', 'on', 'different', 'projects', 'which', 'need', 'python,', 'cybersecurity,', 'HTML,', 'CSS,', 'javascript,', 'java,', 'and', 'Docker', 'containers', 'as', 'main', 'skills'], 'ner_tags': ['B-NAME', 'I-NAME', 'B-QUALIFICATION', 'B-SKILLS', 'I-SKILLS', 'B-SKILLS', 'I-SKILLS', 'B-SKILLS', 'I-SKILLS', 'B-SKILLS', 'I-SKILLS', 'B-SKILLS', 'B-SKILLS', 'B-SKILLS', 'B-SKILLS', 'B-SKILLS', 'B-SKILLS', 'B-SKILLS', 'B-SKILLS', 'B-SKILLS', 'I-SKILLS', 'B-SKILLS', 'B-SKILLS', 'B-SKILLS', 'B-EXPERIENCE', 'I-EXPERIENCE', 'I-EXPERIENCE', 'I-EXPERIENCE', 'I-EXPERIENCE', 'I-EXPERIENCE', 'I-EXPE

In [ ]:
bio_tags = ["O", "B-NAME", "I-NAME", "B-QUALIFICATION", "I-QUALIFICATION", "B-SKILLS", "I-SKILLS", "B-EXPERIENCE", "I-EXPERIENCE"]

In [ ]:
i=0
invalids = []
print(len(all_data))
for j, pair_dict in enumerate(all_data):
  tags = pair_dict['ner_tags']
  valid = True
  for tag in tags:
    if tag not in bio_tags:
      i+=1
      valid = False
      break
  if not valid:
    del all_data[j]
    invalids.append(pair_dict)
    # print(j)
print(i)
print(len(all_data))

4044
0
4044


In [ ]:
for invalid_dict in invalids:
  for i, token in enumerate(invalid_dict['tokens']):
    if token in bio_tags:
      t = invalid_dict['tokens'][i]
      invalid_dict['tokens'][i] = invalid_dict['ner_tags'][i]
      invalid_dict['ner_tags'][i] = t
all_data = all_data + invalids

In [ ]:
## BIO TAG

import pandas as pd
import json

# Function to apply BIO tagging
def bio_tagging(phrases, tag):
    """
    Apply BIO tagging to a list of phrases with a given tag.

    Args:
        phrases (list): List of phrases to tag.
        tag (str): The tag for the entity type.

    Returns:
        list: Tagged words in BIO format as strings.
    """
    tagged_words = []
    for phrase in phrases:
        try:
          words = phrase.split()
          if words:
              tagged_words.append(f"{words[0]} B-{tag}")  # First word gets B-tag
              tagged_words.extend([f"{word} I-{tag}" for word in words[1:]])  # Remaining words get I-tags
        except:
          print(phrases)
    return tagged_words

# Function to add BIO-tagged data as a new column with new lines
def add_bio_tagging(data_frame):
    """
    Process the 'final_annotation' column to generate BIO-tagged output,
    and add it as a new column named 'bio_tag' in the DataFrame,
    with each entity starting on a new line.

    Args:
        data_frame (DataFrame): The input DataFrame with annotations.

    Returns:
        DataFrame: The updated DataFrame with the 'bio_tag' column added.
    """
    bio_tagged_column = []

    for _, row in data_frame.iterrows():
        try:
            annotation = json.loads(row["final_annotation"])  # Parse JSON data
            tagged_lines = []

            # Process Name
            if "Name" in annotation:
                tagged_lines.extend(bio_tagging([annotation["Name"]], "NAME"))

            # Process Qualifications
            if "Qualifications" in annotation:
                tagged_lines.extend(bio_tagging(annotation["Qualifications"], "QUALIFICATION"))

            # Process Skills
            if "Skills" in annotation:
                tagged_lines.extend(bio_tagging(annotation["Skills"], "SKILLS"))

            # Process Experience
            if "Experience" in annotation:
                tagged_lines.extend(bio_tagging(annotation["Experience"], "EXPERIENCE"))

            # Combine tagged lines into a single string separated by newlines
            bio_tagged_column.append("\n".join(tagged_lines))
        except (json.JSONDecodeError, TypeError):
            bio_tagged_column.append("")  # Add an empty string for malformed data

    # Add the new column to the DataFrame
    data_frame["bio_tag"] = bio_tagged_column
    return data_frame

# Main block
if __name__ == "__main__":
    # File paths
    input_file = r"/content/final_resume_annotation_procd.csv"
    output_file = r"/content/bio_tagged_output_with_new_lines.csv"

    # Load the CSV file
    df = pd.read_csv(input_file)

    # Add BIO-tagged data to the DataFrame
    updated_df = add_bio_tagging(df)

    # Save the updated DataFrame to a CSV file
    updated_df.to_csv(output_file, index=False)

    print(f"BIO-tagged data added in 'bio_tag' column (with new lines) and saved to {output_file}")


In [ ]:
from sklearn.model_selection import train_test_split

# Split the full dataset into train + temp (val + test)
train_data, temp_data = train_test_split(all_data, test_size=0.2, random_state=42)

# Split temp into val and test
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)


# **BERT Fine Tuning**

In [ ]:
!pip install datasets

import pickle

with open('data.pkl', 'rb') as file:
    all_data = pickle.load(file)

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Get all unique tags
all_tags = set(tag for example in all_data for tag in example['ner_tags'])

label_list = sorted(list(all_tags))
label_encoder = LabelEncoder()
label_encoder.fit(label_list)

# Map tags to IDs
for example in all_data:
    example['label_ids'] = label_encoder.transform(example['ner_tags'])


In [ ]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

def tokenize_and_align_labels(example):
    tokenized_inputs = tokenizer(example['tokens'], is_split_into_words=True, truncation=True, padding='max_length', max_length=128)

    labels = []
    word_ids = tokenized_inputs.word_ids()
    previous_word_idx = None
    for word_idx in word_ids:
        if word_idx is None:
            labels.append(-100)  # Ignored in loss
        elif word_idx != previous_word_idx:
            labels.append(example['label_ids'][word_idx])
        else:
            labels.append(example['label_ids'][word_idx] if example['ner_tags'][word_idx].startswith("I-") else -100)
        previous_word_idx = word_idx
    tokenized_inputs["labels"] = labels
    return tokenized_inputs


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
from datasets import Dataset
tokenized_dataset = [tokenize_and_align_labels(example) for example in all_data]

hf_dataset = Dataset.from_list(tokenized_dataset)

In [ ]:
dataset = hf_dataset.train_test_split(test_size=0.2)


In [ ]:
token = ""

In [ ]:
from huggingface_hub import login
login(token=token)  # Will prompt you to paste your HF token


In [ ]:
from transformers import BertForTokenClassification

model = BertForTokenClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=len(label_list),
    id2label={i: label for i, label in enumerate(label_list)},
    label2id={label: i for i, label in enumerate(label_list)},
)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    "bert-resume-ner",
    #evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
)


trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-46-0206bba4d830>:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
results = trainer.evaluate()
print(results)


{'eval_loss': 2.1830646991729736, 'eval_model_preparation_time': 0.0031, 'eval_precision': 0.10499111383580305, 'eval_recall': 0.1252377935231343, 'eval_f1': 0.08137403791318396, 'eval_runtime': 5.9189, 'eval_samples_per_second': 136.681, 'eval_steps_per_second': 8.617}


In [ ]:
pip install --upgrade transformers


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [ ]:
import wandb
api_key = ""
wandb.login(key=api_key)  # You’ll be prompted to paste your API key

wandb.init(project="bert-resume-ner")  # Optional: name your project



wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [ ]:
trainer.train()

Step,Training Loss


In [ ]:
trainer.save_model("resume_ner_model")
trainer.model.save_pretrained("resume_ner_model")

In [ ]:
from transformers import BertForTokenClassification, BertTokenizer

model = BertForTokenClassification.from_pretrained("resume_ner_model")
tokenizer = BertTokenizer.from_pretrained("resume_ner_model")

In [ ]:
import torch
def predict_entities(text):
    model.eval()

    # Move model to same device as tokenizer inputs
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Tokenize input
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}  # Move inputs to the same device

    with torch.no_grad():
        outputs = model(**inputs)

    predictions = torch.argmax(outputs.logits, dim=2)
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    labels = [label_list[p.item()] for p in predictions[0]]

    return list(zip(tokens, labels))

predict_entities("Anshu Kumari - Experienced Java developer with Docker and HTML. GoogleSoft (2020-2024) NewYorkpur, Chandashi, USA")



In [ ]:
jd = df.iloc[0]['resume_text']

In [ ]:
from IPython.display import HTML, display

def set_css():
    display(HTML('''
        <style>
            pre { white-space: pre-wrap; }
        </style>
    '''))

get_ipython().events.register('pre_run_cell', set_css)

# PRESENTATION

In [ ]:
# @markdown # **FINAL**

import json
import string
import nltk
from nltk.corpus import stopwords

nltk.download("stopwords")
stop_words = set(stopwords.words("english"))

# -----------------------------------
# Merge subword tokens only
# -----------------------------------
def merge_entities(predictions):
    """
    Given a list of (token, label) predictions, merge subword tokens (those starting with "##")
    with their immediate previous token. Only merge contiguous subwords, not all tokens with the same label.
    Returns three sets: skills, eligibility, and experience.
    """
    merged_skills = set()
    merged_eligibility = set()
    merged_experience = set()

    prev_token = ""
    prev_label = ""

    for token, label in predictions:
        if token in ["[CLS]", "[SEP]"]:
            continue
        label = label.upper()
        # If token is a subword, merge with the previous token
        if token.startswith("##"):
            prev_token += token[2:]
        else:
            # Finalize previous token (if exists)
            if prev_token and prev_label:
                if "SKILL" in prev_label:
                    merged_skills.add(prev_token.lower())
                elif "QUALIFICATION" in prev_label or "ELIGIBILITY" in prev_label:
                    merged_eligibility.add(prev_token.lower())
                elif "EXPERIENCE" in prev_label:
                    merged_experience.add(prev_token.lower())
            # Start a new token group
            prev_token = token
            prev_label = label
    # Final flush
    if prev_token and prev_label:
        if "SKILL" in prev_label:
            merged_skills.add(prev_token.lower())
        elif "QUALIFICATION" in prev_label or "ELIGIBILITY" in prev_label:
            merged_eligibility.add(prev_token.lower())
        elif "EXPERIENCE" in prev_label:
            merged_experience.add(prev_token.lower())

    return merged_skills, merged_eligibility, merged_experience

# -----------------------------------
# Extract JD entities using NER, after cleaning JD text
# -----------------------------------
def extract_jd_entities_ner(jd_text):
    # Preprocess: remove stopwords & punctuation before running NER
    words = jd_text.split()
    cleaned_words = [
        word.strip(string.punctuation)
        for word in words
        if word.lower() not in stop_words and word.strip(string.punctuation)
    ]
    cleaned_text = " ".join(cleaned_words)

    predictions = predict_entities(cleaned_text)
    return merge_entities(predictions)

# -----------------------------------
# Extract entities from a resume entry (from its tokens & ner_tags)
# -----------------------------------
def extract_resume_entities(entry):
    tokens = entry.get("tokens", [])
    ner_tags = entry.get("ner_tags", [])
    skills = set()
    eligibility = set()
    experience = set()

    prev_token = ""
    prev_label = ""

    for token, label in zip(tokens, ner_tags):
        label = label.upper()
        # Merge subword tokens if token starts with "##"
        if token.startswith("##"):
            prev_token += token[2:]
        else:
            if prev_token and prev_label:
                if "SKILL" in prev_label:
                    skills.add(prev_token.lower())
                elif "QUALIFICATION" in prev_label or "ELIGIBILITY" in prev_label:
                    eligibility.add(prev_token.lower())
                elif "EXPERIENCE" in prev_label:
                    experience.add(prev_token.lower())
            prev_token = token
            prev_label = label
    # Final flush:
    if prev_token and prev_label:
        if "SKILL" in prev_label:
            skills.add(prev_token.lower())
        elif "QUALIFICATION" in prev_label or "ELIGIBILITY" in prev_label:
            eligibility.add(prev_token.lower())
        elif "EXPERIENCE" in prev_label:
            experience.add(prev_token.lower())

    return skills, eligibility, experience

# -----------------------------------
# Compute matching score including experience
# -----------------------------------
def compute_match_score(resume_skills, resume_elig, resume_exp, jd_skills, jd_elig, jd_exp):
    matched_skills = resume_skills.intersection(jd_skills)
    matched_eligibility = resume_elig.intersection(jd_elig)
    matched_experience = resume_exp.intersection(jd_exp)
    score = len(matched_skills) + 2 * len(matched_eligibility) + len(matched_experience)
    return score, matched_skills, matched_eligibility, matched_experience

# -----------------------------------
# Resume matching function
# -----------------------------------
def match_resumes_with_jd(jd_text, data):
    jd_skills, jd_elig, jd_exp = extract_jd_entities_ner(jd_text)

    best_score = -1
    best_final_annotation = None
    best_details = None

    for entry in data:
        resume_skills, resume_elig, resume_exp = extract_resume_entities(entry)
        score, matched_skills, matched_eligibility, matched_experience = compute_match_score(
            resume_skills, resume_elig, resume_exp, jd_skills, jd_elig, jd_exp
        )
        if score > best_score:
            best_score = score
            best_final_annotation = entry.get("final_annotation", "No annotation provided")
            best_details = (matched_skills, matched_eligibility, matched_experience)

    return best_score, best_details, best_final_annotation

# -----------------------------------
# Pretty printing a resume (for demonstration)
# -----------------------------------
def print_resume(data):
    print(f"{'='*60}")
    print(f"📄 Resume: {data.get('Name', 'Unknown')}")
    print(f"{'='*60}\n")

    print("🎓 Qualifications:")
    for qual in data.get("Qualifications", []):
        print(f"  • {qual}")
    print()

    print("🛠️ Skills:")
    for skill in data.get("Skills", []):
        print(f"  • {skill}")
    print()

    print("💼 Experience:")
    for exp in data.get("Experience", []):
        print(f"  • {exp}\n")
    print(f"{'='*60}")

# -----------------------------------
# Sample Job Descriptions to test
# -----------------------------------
# -----------------------------------
# Assuming your resume data is in variable 'data'
# 'data' should be a list of dictionaries, each with keys:
# 'tokens', 'ner_tags', and 'final_annotation' (which is a JSON string of the full resume details)
# For demonstration, here's a dummy example resume data:

# -----------------------------------
# Run Matching for Each JD and Print Results
# -----------------------------------


def predict_entities(text):
    model.eval()

    # Move model to same device as tokenizer inputs
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Tokenize input
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}  # Move inputs to the same device

    with torch.no_grad():
        outputs = model(**inputs)

    predictions = torch.argmax(outputs.logits, dim=2)
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    labels = [label_list[p.item()] for p in predictions[0]]

    return list(zip(tokens, labels))



def run(jds):
  for jd in jds:
      print("-----------------------------------------------------")
      print("Job Description:")
      print(jd)
      print()
      data = all_data
      best_score, best_details, final_annotation = match_resumes_with_jd(jd, data)

      print("✅ Best Match Score:", best_score)
      if best_details:
          matched_skills, matched_elig, matched_exp = best_details
          print("🎯 Matched Skills:", sorted(matched_skills))
          print("🎓 Matched Eligibility:", sorted(matched_elig))
          print("💼 Matched Experience:", sorted(matched_exp))
      else:
          print("No matched details.")

      print("\n📎 Final Annotation of Best Matching Resume:")
      try:
          final_annotation_dict = json.loads(final_annotation)
          print_resume(final_annotation_dict)
      except Exception as e:
          print("⚠️ Error parsing final annotation:", e)
          print(final_annotation)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
jds = [
    """Software Engineer:
As a Software Engineer, you will be responsible for designing, developing, and maintaining software applications that meet business needs. You will collaborate with cross-functional teams, including product managers, designers, and other engineers, to deliver high-quality solutions. Your role will involve writing clean, efficient, and scalable code, conducting code reviews, and troubleshooting software issues. You will also be expected to stay updated with the latest technologies and best practices in software development.

Requirements:
- Bachelor's degree in Computer Science, Software Engineering, or a related field
- Proficiency in programming languages such as Python, Java, or C++
- Experience with software development lifecycle, including Agile methodologies
- Strong problem-solving skills and ability to debug complex issues
- Knowledge of cloud computing platforms such as AWS, Azure, or Google Cloud
- Excellent communication and teamwork skills""",

    """Data Analyst:
As a Data Analyst, you will be responsible for collecting, processing, and analyzing large datasets to extract meaningful insights that drive business decisions. You will develop reports, dashboards, and visualizations to communicate findings effectively. Your role will involve working closely with stakeholders to understand their data needs and provide actionable recommendations. You will also ensure data integrity and accuracy by implementing data validation techniques.

Requirements:
- Strong analytical and problem-solving skills
- Experience with SQL for querying databases and data manipulation
- Proficiency in data visualization tools such as Tableau, Power BI, or Excel
- Bachelor's degree in Statistics, Mathematics, Data Science, or a related field
- Knowledge of machine learning concepts and predictive analytics is a plus
- Ability to communicate complex data findings in a clear and concise manner""",

    """Cybersecurity Specialist:
As a Cybersecurity Specialist, you will be responsible for implementing security measures to protect systems, networks, and data from cyber threats. You will monitor security incidents, conduct vulnerability assessments, and respond to security breaches. Your role will involve developing security policies, training employees on cybersecurity best practices, and ensuring compliance with industry regulations.

Requirements:
- Knowledge of cybersecurity frameworks such as NIST, ISO 27001, or CIS
- Experience with penetration testing, ethical hacking, and vulnerability assessment
- Certifications such as CISSP, CEH, or CompTIA Security+ preferred
- Strong understanding of encryption, firewalls, and intrusion detection systems
- Bachelor's degree in Cybersecurity, Computer Science, or a related field
- Ability to work under pressure and handle security incidents effectively""",

    """Cloud Engineer:
As a Cloud Engineer, you will design, implement, and manage cloud infrastructure solutions that optimize performance, security, and cost efficiency. You will work with cloud service providers such as AWS, Azure, or Google Cloud to deploy scalable applications. Your role will involve automating cloud operations, monitoring system performance, and troubleshooting issues.

Requirements:
- Experience with cloud computing platforms (AWS, Azure, Google Cloud)
- Knowledge of containerization and orchestration tools such as Docker and Kubernetes
- Strong scripting skills in Python, Bash, or PowerShell
- Bachelor's degree in Computer Science, Cloud Computing, or a related field
- Understanding of cloud security best practices and compliance standards
- Ability to optimize cloud resources for cost efficiency""",

    """DevOps Engineer:
As a DevOps Engineer, you will be responsible for automating deployment processes, improving system reliability, and ensuring seamless collaboration between development and operations teams. You will implement CI/CD pipelines, monitor system performance, and troubleshoot infrastructure issues.

Requirements:
- Experience with CI/CD tools such as Jenkins, GitLab CI/CD, or CircleCI
- Knowledge of scripting languages such as Python, Bash, or Ruby
- Familiarity with Kubernetes, Docker, and cloud infrastructure
- Strong problem-solving skills and ability to optimize deployment workflows
- Bachelor's degree in Computer Science, DevOps, or a related field
- Excellent communication and collaboration skills""",

    """IT Support Specialist:
As an IT Support Specialist, you will provide technical assistance to end-users, troubleshoot hardware and software issues, and ensure smooth IT operations within the organization. You will install and configure computer systems, respond to service requests, and maintain IT documentation.

Requirements:
- Strong problem-solving skills and ability to diagnose technical issues
- Experience with Windows, Linux, and macOS operating systems
- Knowledge of networking fundamentals and troubleshooting techniques
- Excellent communication and customer service skills
- Bachelor's degree in Information Technology or a related field
- Ability to work in a fast-paced environment and prioritize tasks effectively""",

    """Database Administrator:
As a Database Administrator, you will manage and optimize database systems to ensure data integrity, security, and performance. You will be responsible for database backup and recovery, monitoring system health, and implementing best practices for data management.

Requirements:
- Experience with SQL and NoSQL databases such as MySQL, PostgreSQL, MongoDB
- Knowledge of database backup, recovery, and replication strategies
- Strong understanding of indexing, query optimization, and performance tuning
- Bachelor's degree in Information Technology, Database Management, or a related field
- Ability to troubleshoot database issues and implement security measures""",

    """Network Engineer:
As a Network Engineer, you will design, configure, and maintain network infrastructure to ensure seamless connectivity and security. You will monitor network performance, troubleshoot issues, and implement security protocols.

Requirements:
- Experience with routing and switching technologies
- Knowledge of network security protocols such as VPN, firewalls, and IDS/IPS
- Certifications such as CCNA, CCNP, or equivalent preferred
- Strong problem-solving skills and ability to optimize network performance
- Bachelor's degree in Computer Networking or a related field""",

    """AI/ML Engineer:
As an AI/ML Engineer, you will develop machine learning models and AI solutions that enhance business processes and decision-making. You will optimize algorithms for performance and accuracy, work with large datasets, and deploy AI applications.

Requirements:
- Experience with Python and ML frameworks such as TensorFlow, PyTorch, or Scikit-learn
- Strong mathematical and statistical skills
- Bachelor's or Master's degree in AI, Data Science, or a related field
- Knowledge of deep learning, NLP, and computer vision techniques""",

    """UI/UX Designer:
As a UI/UX Designer, you will create user-friendly interfaces and improve user experience through research, design, and usability testing. You will collaborate with developers and stakeholders to ensure intuitive and visually appealing designs.

Requirements:
- Proficiency in design tools such as Figma, Adobe XD, or Sketch
- Experience with front-end technologies like HTML, CSS, and JavaScript
- Strong understanding of user-centered design principles
- Bachelor's degree in Graphic Design, UI/UX, or a related field
- Ability to conduct usability testing and iterate designs based on feedback"""
]

run(jds)

-----------------------------------------------------
Job Description:
Software Engineer:
As a Software Engineer, you will be responsible for designing, developing, and maintaining software applications that meet business needs. You will collaborate with cross-functional teams, including product managers, designers, and other engineers, to deliver high-quality solutions. Your role will involve writing clean, efficient, and scalable code, conducting code reviews, and troubleshooting software issues. You will also be expected to stay updated with the latest technologies and best practices in software development.

Requirements:
- Bachelor's degree in Computer Science, Software Engineering, or a related field
- Proficiency in programming languages such as Python, Java, or C++
- Experience with software development lifecycle, including Agile methodologies
- Strong problem-solving skills and ability to debug complex issues
- Knowledge of cloud computing platforms such as AWS, Azure, or Goog

In [ ]:
# FINNAL ANNOTATION FROM ANNOTATION COLUMN


# Function to process each row and extract relevant information
def extract_final_annotation(annotation):
    try:
        if pd.isna(annotation):  # Skip empty or NaN annotations
            return None

        annotation = json.loads(annotation)  # Convert string to dictionary

        # Extract fields safely
        # name = annotation.get("name", "").strip()
        name = annotation.get("name", "")
        name = str(name).strip() if name else ""

        # Process qualifications (Ensure it's a list)
        qualifications = annotation.get("qualifications", [])
        if isinstance(qualifications, dict):
            qualifications = [f"{k}: {v}" for k, v in qualifications.items()]
        elif not isinstance(qualifications, list):
            qualifications = [str(qualifications)]  # Ensure it's a list of strings

        # Process skills (Ensure it's a list)
        skills = annotation.get("skills", [])
        if isinstance(skills, dict):
            skills = [f"{k}: {v}" for k, v in skills.items()]
        elif not isinstance(skills, list):
            skills = [str(skills)]  # Ensure it's a list of strings

        # Process experience (Ensure it's a list)
        # Process experience dynamically without hardcoding missing values
        experience_data = annotation.get("experience", [])
        experience = []
        for exp in experience_data:
            if isinstance(exp, dict):
                exp_details = [str(value) for key, value in exp.items() if value]  # Collect non-empty fields
                if exp_details:  # Only add if there's valid data
                    experience.append(", ".join(exp_details))
            else:
                experience.append(str(exp))  # If it's already a string, add as is


        final_annotation_dict = {
            "Name": name,
            "Qualifications": qualifications,
            "Skills": skills,
            "Experience": experience
        }

        return json.dumps(final_annotation_dict, ensure_ascii=False)  # Convert to JSON string

    except json.JSONDecodeError:
        return None  # Skip row if JSON parsing fails

# Apply function to each row and store results in a new column
df["final_annotation"] = df["annotations"].apply(extract_final_annotation)



# Save the updated CSV file
output_path = "/content/final_resume_annotation_procd.csv"
df.to_csv(output_path, index=False)

print(f"Updated file saved at: {output_path}")



Updated file saved at: /content/final_resume_annotation_procd.csv


In [ ]:
df[df['bio_tag'].isna()]

,json_file_name,resume_text,annotations,processed_annotations,final_annotation,bio_tag


In [ ]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report

def compute_metrics(p):
    predictions, labels = p
    predictions = predictions.argmax(axis=-1)

    true_predictions = []
    true_labels = []

    for pred, label in zip(predictions, labels):
        for p_i, l_i in zip(pred, label):
            if l_i != -100:  # ignore padding
                true_predictions.append(p_i)
                true_labels.append(l_i)

    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, true_predictions, average="macro")

    return {
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }
